In [1]:
import numpy as np
from multiple_factor import DGP2, Inferece2
from dgp import DGP

def reject_prob(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    phi_tau = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        inf = Inferece2(Y, D, tuple_idx, design)
        phi_tau[i] = inf.phi_tau
    return np.mean(phi_tau)


def risk(Xdim, num_factor, sample_size, tau=0, ntrials=1000, more=False, design='MT'):
    mse = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim, tau, more, design)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_idx
        ate = np.mean(Y[D[:,0]==1]) - np.mean(Y[D[:,0]==0])
        mse[i] = (ate - tau)**2
    return np.mean(mse)


def test(q,k):
    print(q,k)
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='S4'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='C'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='RE'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MP-B'))
    print(risk(q, k, 1280, tau=0, ntrials=100, more=False, design='MT'))

#[test(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
#test(1,2)
#test(2,2)
#test(10,6)

In [2]:
print([(q,k,reject_prob(q,k,1280,tau=0,ntrials=1000,more=False, design='C')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.061), (1, 2, 0.068), (1, 3, 0.071), (1, 4, 0.049), (1, 5, 0.063), (1, 6, 0.046), (2, 1, 0.047), (2, 2, 0.046), (2, 3, 0.051), (2, 4, 0.065), (2, 5, 0.056), (2, 6, 0.059), (4, 1, 0.055), (4, 2, 0.048), (4, 3, 0.041), (4, 4, 0.055), (4, 5, 0.064), (4, 6, 0.053), (8, 1, 0.048), (8, 2, 0.064), (8, 3, 0.052), (8, 4, 0.052), (8, 5, 0.057), (8, 6, 0.056), (10, 1, 0.045), (10, 2, 0.069), (10, 3, 0.047), (10, 4, 0.049), (10, 5, 0.05), (10, 6, 0.072)]


In [3]:
print([(q,k,reject_prob(q,k,1280,tau=0.02,ntrials=1000,more=False, design='C')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.226), (1, 2, 0.225), (1, 3, 0.205), (1, 4, 0.201), (1, 5, 0.242), (1, 6, 0.243), (2, 1, 0.08), (2, 2, 0.089), (2, 3, 0.072), (2, 4, 0.09), (2, 5, 0.093), (2, 6, 0.086), (4, 1, 0.067), (4, 2, 0.075), (4, 3, 0.08), (4, 4, 0.08), (4, 5, 0.073), (4, 6, 0.076), (8, 1, 0.082), (8, 2, 0.071), (8, 3, 0.072), (8, 4, 0.06), (8, 5, 0.065), (8, 6, 0.065), (10, 1, 0.057), (10, 2, 0.053), (10, 3, 0.054), (10, 4, 0.057), (10, 5, 0.065), (10, 6, 0.066)]


In [2]:
print([(q,k,reject_prob(q,k,1280,tau=0,ntrials=1000,more=False, design='S4')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.043), (1, 2, 0.057), (1, 3, 0.05), (1, 4, 0.061), (1, 5, 0.064), (1, 6, 0.068), (2, 1, 0.046), (2, 2, 0.053), (2, 3, 0.051), (2, 4, 0.055), (2, 5, 0.067), (2, 6, 0.078), (4, 1, 0.043), (4, 2, 0.053), (4, 3, 0.064), (4, 4, 0.053), (4, 5, 0.059), (4, 6, 0.09), (8, 1, 0.049), (8, 2, 0.053), (8, 3, 0.054), (8, 4, 0.052), (8, 5, 0.068), (8, 6, 0.068), (10, 1, 0.046), (10, 2, 0.051), (10, 3, 0.047), (10, 4, 0.058), (10, 5, 0.06), (10, 6, 0.092)]


In [3]:
print([(q,k,reject_prob(q,k,1280,tau=0.02,ntrials=1000,more=False, design='S4')) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]])

[(1, 1, 0.836), (1, 2, 0.828), (1, 3, 0.819), (1, 4, 0.833), (1, 5, 0.843), (1, 6, 0.881), (2, 1, 0.167), (2, 2, 0.17), (2, 3, 0.188), (2, 4, 0.194), (2, 5, 0.197), (2, 6, 0.246), (4, 1, 0.063), (4, 2, 0.083), (4, 3, 0.074), (4, 4, 0.09), (4, 5, 0.102), (4, 6, 0.12), (8, 1, 0.057), (8, 2, 0.063), (8, 3, 0.068), (8, 4, 0.078), (8, 5, 0.074), (8, 6, 0.09), (10, 1, 0.055), (10, 2, 0.071), (10, 3, 0.062), (10, 4, 0.065), (10, 5, 0.067), (10, 6, 0.09)]


designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results = []

for m in designs:
    print(m)
    if m == 'MT2':
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT') 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    else:
        result = {(q,k): risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
         for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]}
    results.append(result)
    baseline = results[0][(1,1)]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.3f} \\\\".format(result[(q,k)]/baseline))

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
designs = ['MT', 'C', 'MT2', 'S4', 'MP-B', 'RE']
results = []

num_cores = multiprocessing.cpu_count()
for m in designs:
    print(m)
    qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [1,2,3,4,5,6]]
    def processInput(qk):
        q, k = qk
        if m == 'MT2':
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=True, design='MT')
        else:
            mse = risk(q, k, 1280, tau=0, ntrials=1000, more=False, design=m) 
        return (q,k,mse)
    ret = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
    result = {(re[0], re[1]):re[2] for re in ret}
    results.append(result)
    baseline = results[0][(1,1)]
    #for q in [1,2,4,8,10]:
    #    for k in [1,2,3,4,5,6]:
    #        if k<6:
    #            print("{:.3f} & ".format(result[(q,k)]/baseline), end = '')
    #        else:
    #            print("{:.3f} \\\\".format(result[(q,k)]/baseline))

In [12]:
for i, m in enumerate(designs):
    print(m)
    baseline = results[0][(1,1)]
    result = results[i]
    for q in [1,2,4,8,10]:
        for k in [1,2,3,4,5,6]:
            if k<6:
                print("{:.2f} & ".format(result[(q,k)]/baseline), end = '')
            else:
                print("{:.2f} \\\\".format(result[(q,k)]/baseline))

MT
1.00 & 0.91 & 1.03 & 0.99 & 1.06 & 0.98 \\
1.08 & 1.08 & 1.21 & 1.60 & 2.08 & 3.53 \\
2.88 & 4.19 & 6.38 & 10.83 & 14.93 & 23.33 \\
18.53 & 27.17 & 43.58 & 55.88 & 72.74 & 99.04 \\
34.63 & 47.50 & 66.16 & 86.31 & 105.05 & 133.20 \\
C
8.72 & 9.91 & 8.94 & 8.69 & 9.16 & 9.37 \\
39.86 & 40.83 & 41.96 & 43.14 & 40.74 & 41.66 \\
82.63 & 80.09 & 77.19 & 76.09 & 77.29 & 75.39 \\
150.79 & 151.07 & 158.89 & 153.51 & 162.58 & 152.25 \\
191.09 & 198.80 & 198.30 & 187.13 & 186.74 & 189.37 \\
MT2
0.95 & 0.96 & 0.93 & 1.03 & 1.03 & 0.91 \\
1.08 & 1.30 & 1.49 & 2.28 & 3.39 & 6.09 \\
4.22 & 6.20 & 10.51 & 15.42 & 22.64 & 32.47 \\
29.52 & 42.20 & 54.75 & 73.48 & 93.41 & 107.66 \\
52.36 & 67.61 & 85.84 & 105.52 & 130.84 & 152.69 \\
S4
1.36 & 1.47 & 1.53 & 1.47 & 1.46 & 1.51 \\
4.73 & 4.78 & 5.15 & 4.57 & 4.36 & 4.69 \\
10.09 & 10.75 & 10.34 & 10.89 & 9.31 & 10.59 \\
20.74 & 20.32 & 22.91 & 21.18 & 21.35 & 22.60 \\
25.63 & 26.80 & 27.10 & 28.01 & 26.58 & 26.32 \\
MP-B
0.93 & 8.43 & 9.29 & 9.12 & 8.95 

MT
1.00 & 0.91 & 1.03 & 0.99 & 1.06 & 0.98 \\
1.08 & 1.08 & 1.21 & 1.60 & 2.08 & 3.53 \\
2.88 & 4.19 & 6.38 & 10.83 & 14.93 & 23.33 \\
18.53 & 27.17 & 43.58 & 55.88 & 72.74 & 99.04 \\
34.63 & 47.50 & 66.16 & 86.31 & 105.05 & 133.20 \\
C
8.72 & 9.91 & 8.94 & 8.69 & 9.16 & 9.37 \\
39.86 & 40.83 & 41.96 & 43.14 & 40.74 & 41.66 \\
82.63 & 80.09 & 77.19 & 76.09 & 77.29 & 75.39 \\
150.79 & 151.07 & 158.89 & 153.51 & 162.58 & 152.25 \\
191.09 & 198.80 & 198.30 & 187.13 & 186.74 & 189.37 \\
MT2
0.95 & 0.96 & 0.93 & 1.03 & 1.03 & 0.91 \\
1.08 & 1.30 & 1.49 & 2.28 & 3.39 & 6.09 \\
4.22 & 6.20 & 10.51 & 15.42 & 22.64 & 32.47 \\
29.52 & 42.20 & 54.75 & 73.48 & 93.41 & 107.66 \\
52.36 & 67.61 & 85.84 & 105.52 & 130.84 & 152.69 \\
S4
1.49 & 1.47 & 1.41 & 1.39 & 1.42 & 1.46 \\
11.14 & 11.21 & 11.40 & 11.07 & 10.81 & 10.50 \\
43.71 & 48.87 & 47.62 & 48.38 & 48.81 & 50.79 \\
126.12 & 116.34 & 133.74 & 127.31 & 120.85 & 129.07 \\
172.83 & 167.25 & 168.09 & 167.19 & 155.21 & 165.91 \\
MP-B
0.93 & 8.43 & 9.29 & 9.12 & 8.95 & 8.82 \\
1.05 & 42.44 & 39.21 & 41.90 & 40.50 & 41.14 \\
2.82 & 76.36 & 79.47 & 80.37 & 80.51 & 77.41 \\
19.70 & 155.50 & 146.03 & 162.59 & 156.69 & 156.09 \\
31.87 & 196.99 & 184.88 & 179.50 & 181.83 & 189.76 \\
RE
1.26 & 9.48 & 8.66 & 9.14 & 8.45 & 9.41 \\
4.47 & 45.67 & 38.90 & 41.42 & 39.33 & 43.90 \\
11.52 & 79.23 & 84.80 & 78.26 & 77.86 & 84.99 \\
36.04 & 153.68 & 152.06 & 144.94 & 154.13 & 143.51 \\
52.83 & 204.49 & 190.11 & 202.37 & 186.89 & 180.32 \\